# โมเสคแบบเลือก mean/max แล้วสามารถเลือก MFB ว่าจะใช้ไม่ใช้
* เลือก mean/max
* เลือกว่าจะปรับแก้ MFB ก่อนโมเสคหรือไม่
* เลือก ZR ที่ต้องการได้ มีให้เลือกสามแบบ

In [10]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''

import os
import csv
import glob
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.transform import from_bounds
from rasterio.crs import CRS
from rasterio.features import rasterize
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import CRS, Transformer, Geod

def get_input_files(base_dir):
    radar_files = {
        'CHN': [], 'CMP': [], 'CRI': [], 'KKN': [], 'KRB': [], 'LMP': [],
        'NRT': [], 'PHS': [], 'SNK': [], 'STP': [], 'SVP': [], 'PKT': []
    }
    
    for radar in radar_files.keys():
        pattern = os.path.join(base_dir, f'00outp_cappi_2km_{radar}', '*.tif')
        files = sorted(glob.glob(pattern))
        radar_files[radar] = files
    
    return radar_files

def generate_mosaic_report(radar_files, output_dir, expected_radars=12):
    report_data = []
    all_timestamps = set()
    
    for radar_list in radar_files.values():
        all_timestamps.update(os.path.basename(f).split('.')[0] for f in radar_list)
    
    all_timestamps = sorted(all_timestamps)
    
    for timestamp in all_timestamps:
        available_radars = [radar for radar, file_list in radar_files.items() 
                            if any(timestamp in f for f in file_list)]
        
        report_data.append({
            'timestamp': timestamp,
            'available_radars': ','.join(available_radars),
            'total_radars': len(available_radars),
            'expected_radars': expected_radars,
            'missing_radars': expected_radars - len(available_radars)
        })
    
    report_file = os.path.join(output_dir, 'mosaic_report.csv')
    with open(report_file, 'w', newline='') as csvfile:
        fieldnames = ['timestamp', 'available_radars', 'total_radars', 'expected_radars', 'missing_radars']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in report_data:
            writer.writerow(row)
    
    print(f"Mosaic report saved to {report_file}")
    return report_data, all_timestamps

def reproject_to_geographic(src, dst_crs=CRS.from_epsg(4326)):
    dst_transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    
    reprojected_data = np.full((height, width), np.nan, dtype=np.float32)
    reproject(
        source=rasterio.band(src, 1),
        destination=reprojected_data,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )
    
    return reprojected_data, dst_transform, dst_crs

def create_geodetic_radar_buffers(radar_dict, active_radars, radius_km, crs):
    radar_data = []
    geod = Geod(ellps="WGS84")
    azimuths = np.linspace(0, 360, 361)
    
    for radar in active_radars:
        info = radar_dict[radar]
        lon, lat, _ = info['coords']
        buffer_points = [geod.fwd(lon, lat, azimuth, radius_km * 1000)[:2] for azimuth in azimuths]
        radar_data.append({'radar': radar, 'geometry': Polygon(buffer_points)})
    
    gdf_radars = gpd.GeoDataFrame(radar_data, crs=crs)
    return gdf_radars

def dbz_to_rainfall(dbz, zr_relation='marshall-palmer'):
    zr_relations = {
        'marshall-palmer': (200, 1.6),
        'rosenfeld-tropical': (250, 1.2),
        'summer-deep-convective': (300, 1.4)
    }
    
    if zr_relation not in zr_relations:
        raise ValueError(f"Unknown Z-R relation: {zr_relation}. Available options are: {', '.join(zr_relations.keys())}")
    
    a, b = zr_relations[zr_relation]
    Z = 10 ** (dbz / 10)  # Convert dBZ to Z
    R = (Z / a) ** (1 / b)
    return R

def load_mfb_values(file_path, threshold='1.0'):
    mfb_values = {}
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row['Threshold'] == threshold:
                radar = row['Radar']
                zr_relation = row['ZR_Relation']
                mfb = float(row['MFB']) if row['MFB'] else 1.0
                
                if radar not in mfb_values:
                    mfb_values[radar] = {}
                mfb_values[radar][zr_relation] = mfb
    #print('mfb_values>>>> threshold : ', threshold, mfb_values)
    return mfb_values

def apply_mfb_adjustment(radar_data, radar_code, zr_relation, mfb_values):
    if radar_code not in mfb_values or zr_relation not in mfb_values[radar_code]:
        print(f"Warning: No MFB value found for radar {radar_code} and Z-R relation {zr_relation}. Using unadjusted data.")
        return radar_data
    
    mfb = mfb_values[radar_code][zr_relation]
    #print('mfb_values[radar_code][zr_relation]>>>>//', radar_code, ' ,', zr_relation, ' ,',  mfb_values[radar_code][zr_relation])
    
    # Apply MFB adjustment only for rainfall values greater than 0
    adjusted_data = np.where(radar_data > 0, radar_data * mfb, radar_data)
    
    return adjusted_data

def calculate_mosaic_extent_geo(reprojected_data):
    #ax.set_xlim(96.15, 106.5)
    #ax.set_ylim(4.25, 22.15)
    
    #all_bounds = [rasterio.transform.array_bounds(data.shape[0], data.shape[1], transform) 
    #              for data, transform, _, _ in reprojected_data]
    
    #left = min(bound[0] for bound in all_bounds)
    #bottom = min(bound[1] for bound in all_bounds)
    #right = max(bound[2] for bound in all_bounds)
    #top = max(bound[3] for bound in all_bounds)
    left = 96.15
    bottom = 4.25
    right = 106.5
    top = 22.15
    
    
    return (left, bottom, right, top)

def calculate_rainfall_mosaic(reprojected_data, resolution, method='mean', zr_relation='marshall-palmer', mfb_values=None, apply_mfb=False):
    mosaic_bounds = calculate_mosaic_extent_geo(reprojected_data)
    width = int((mosaic_bounds[2] - mosaic_bounds[0]) / resolution)
    height = int((mosaic_bounds[3] - mosaic_bounds[1]) / resolution)
    dst_transform = from_bounds(*mosaic_bounds, width, height)

    if method == 'mean':
        summed_rainfall_data = np.zeros((height, width), dtype=np.float64)
        count_valid_data = np.zeros((height, width), dtype=np.float64)
    elif method == 'max':
        max_rainfall_data = np.full((height, width), -np.inf, dtype=np.float64)
    else:
        raise ValueError("Method must be either 'mean' or 'max'")

    combined_mask = np.zeros((height, width), dtype=bool)

    for data, src_transform, _, radar_code in reprojected_data:
        resampled = np.full((height, width), np.nan, dtype=np.float32)
        reproject(
            source=data,
            destination=resampled,
            src_transform=src_transform,
            dst_transform=dst_transform,
            src_crs=CRS.from_epsg(4326),
            dst_crs=CRS.from_epsg(4326),
            resampling=Resampling.nearest
        )
        rainfall_data = dbz_to_rainfall(resampled, zr_relation)
        
        if apply_mfb and mfb_values is not None:
            rainfall_data = apply_mfb_adjustment(rainfall_data, radar_code, zr_relation, mfb_values)

        radar_mask = ~np.isnan(rainfall_data)
        combined_mask |= radar_mask

        if method == 'mean':
            summed_rainfall_data = np.nansum([summed_rainfall_data, rainfall_data], axis=0)
            count_valid_data += radar_mask.astype(np.float64)
        elif method == 'max':
            max_rainfall_data = np.fmax(max_rainfall_data, rainfall_data)

    if method == 'mean':
        result_rainfall_data = np.divide(summed_rainfall_data, count_valid_data, where=count_valid_data > 0)
    elif method == 'max':
        result_rainfall_data = np.where(max_rainfall_data == -np.inf, np.nan, max_rainfall_data)

    return result_rainfall_data, dst_transform, CRS.from_epsg(4326)

def apply_threshold_and_mask(data, radar_mask, threshold=0.1):
    result = np.full_like(data, np.nan, dtype=np.float32)
    
    mask_within_coverage = ~np.isnan(radar_mask)
    
    result[mask_within_coverage] = np.where(data[mask_within_coverage] >= threshold, 
                                            data[mask_within_coverage], 
                                            0)
    
    return result

def save_geotiff(output_file, data, transform, crs):
    with rasterio.open(
        output_file, 'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(data, 1)

def plot_rainfall_map(data, radar_mask, transform, crs, basin_shapefile, output_dir, time_hr, vmin=0, vmax=5, filename_suffix=''):
    """
    Plot rainfall estimate map with jet colormap, grey for 0 mm inside radar coverage,
    and white outside radar coverage.
    """
    # Create a custom colormap
    cmap = plt.cm.jet
    n_bins = 256
    jet_colors = cmap(np.linspace(0, 1, n_bins))
    
    # Set the first color (for 0 mm) to grey
    jet_colors[0] = [0.5, 0.5, 0.5, 1]
    
    custom_cmap = ListedColormap(jet_colors)
    custom_cmap.set_bad('white')  # For NaN values (outside radar mask)

    # Create a masked array for the rainfall data
    masked_data = np.ma.masked_where(np.isnan(radar_mask), data)

    # Plotting the data
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Define the extent for the plot
    extent = rasterio.transform.array_bounds(data.shape[0], data.shape[1], transform)
    extent = [extent[0], extent[2], extent[1], extent[3]]  # [min_lon, max_lon, min_lat, max_lat]

    # Plot the rainfall data
    im = ax.imshow(masked_data, cmap=custom_cmap, extent=extent, origin='upper', 
                   vmin=vmin, vmax=vmax)

    # Load and plot the basin shapefile
    gdf = gpd.read_file(basin_shapefile)
    gdf = gdf.to_crs(crs)
    gdf.boundary.plot(ax=ax, color='black', linewidth=1)

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, extend='max')
    cbar.set_label('Rainfall Intensity (mm/hr)')

    ax.set_title(f'Radar Rainfall Estimate - {time_hr}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Set extent to the specified values
    ax.set_xlim(96.15, 106.5)
    ax.set_ylim(4.25, 22.15)
    
    plt.tight_layout()
    
    # Save plot
    plt.savefig(f'{output_dir}/{time_hr}_rainfall{filename_suffix}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up memory

'''    
def plot_radar_coverage_mask(radar_mask, transform, output_dir, filename='radar_coverage_mask.png'):
    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Define the extent for the plot
    extent = rasterio.transform.array_bounds(radar_mask.shape[0], radar_mask.shape[1], transform)
    extent = [extent[0], extent[2], extent[1], extent[3]]

    # Plot the radar coverage mask
    im = ax.imshow(~np.isnan(radar_mask), cmap='binary', extent=extent, origin='upper')

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Radar Coverage')
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels(['No Coverage', 'Coverage'])

    # Set title and labels
    ax.set_title('Radar Coverage Mask')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Set plot limits (adjust these as needed to match your data)
    ax.set_xlim(96.15, 106.5)
    ax.set_ylim(4.25, 22.15)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/{filename}', dpi=300, bbox_inches='tight')
    plt.show(fig)  # Close the figure to free up memory

    print(f"Radar coverage mask plot saved to {output_dir}/{filename}")    
'''
def main(mosaic_method='mean', apply_mfb=True):
    #base_dir = '../3output/0mosaic/0ztest/0processing_single_cappi/'
    base_dir = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0processing_single_cappi/'
    
    # Define output directories based on the mosaic method
    if mosaic_method == 'mean':
        output_dir_mos = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/1mean_mos/0mosaics'
        mosaic_num = '1'
    elif mosaic_method == 'max':
        output_dir_mos = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/2max_mos/0mosaics'
        mosaic_num = '2'

    output_dir_mos_mfb = os.path.join(output_dir_mos, '0mfb')
    output_dir_mos_no_mfb = os.path.join(output_dir_mos, '0no_mfb')

    # Here we add the number before mosaic_method using `mosaic_num`
    output_dir_map = f'../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/{mosaic_nuwfm}{mosaic_method}_mos/1map_plot'

    # Create necessary directories if they don't exist
    os.makedirs(output_dir_mos_mfb, exist_ok=True)
    os.makedirs(output_dir_mos_no_mfb, exist_ok=True)
    os.makedirs(output_dir_map, exist_ok=True)  # This ensures that the map plot directory is created
   
    resolution = 0.02  # Resolution in degrees
    
    radar_files = get_input_files(base_dir)
    basin_shapefile = '../1data/1GIS/0base_map_gis/MainBasin_ONWR_Law_WGS84Geo.shp'
    
    all_radars = {
        "CHN": {'coords': (100.191263, 15.157852, 40.0), 'el': 0.5},
        "CMP": {'coords': (99.188203, 10.493099, 32.0), 'el': 0.0},
        "CRI": {'coords': (99.881593, 19.961471, 444.0), 'el': 1.08},
        "KKN": {'coords': (102.785881, 16.4625, 217.0), 'el': 0.47},
        "KRB": {'coords': (98.97806, 8.101389, 52.0), 'el': 0.8},
        "LMP": {'coords': (99.041701, 18.565399, 320.0), 'el': 0.48},
        "NRT": {'coords': (101.825165, 6.426888, 33.0), 'el': 0.5},
        "PHS": {'coords': (100.217964, 16.775408, 72.0), 'el': 0.47},
        "PKT": {'coords': (98.329444, 8.133611, 281.0), 'el': 0.08},
        "SNK": {'coords': (104.132591, 17.156363, 196.0), 'el': 0.47},
        "STP": {'coords': (100.459996, 7.449996, 33.0), 'el': 0.5},
        "SVP": {'coords': (100.7675, 13.686389, 28.0), 'el': 0.7},
    }

    report_data, all_timestamps = generate_mosaic_report(radar_files, output_dir_mos, expected_radars=12)

    mfb_values = None
    if apply_mfb:
        mfb_file_path = './0Zprocessing_data/0analysis_results/combined_results_threshold_1.0.csv'
        mfb_values = load_mfb_values(mfb_file_path)

    for time_hr in all_timestamps:
        print(f"Processing time: {time_hr}")
        try:
            current_radar_files = {
                radar: next((f for f in files if time_hr in f), None)
                for radar, files in radar_files.items()
            }
            current_radar_files = {k: v for k, v in current_radar_files.items() if v is not None}

            if not current_radar_files:
                print(f"No radar files available for {time_hr}. Skipping.")
                continue

            active_radars = list(current_radar_files.keys())

            reprojected_data = []
            for radar, radar_file in current_radar_files.items():
                with rasterio.open(radar_file) as src:
                    data, transform, crs = reproject_to_geographic(src)
                    reprojected_data.append((data, transform, crs, radar))

            rainfall_mosaic, transform, crs = calculate_rainfall_mosaic(
                reprojected_data, resolution, method=mosaic_method, 
                zr_relation='rosenfeld-tropical', mfb_values=mfb_values, apply_mfb=apply_mfb
            )

            radar_buffers_geo = create_geodetic_radar_buffers(all_radars, active_radars, 240, "EPSG:4326")

            radar_mask = rasterize(
                [(geom, 1) for geom in radar_buffers_geo.geometry],
                out_shape=rainfall_mosaic.shape,
                transform=transform,
                fill=np.nan,
                dtype=np.float32
            )
            # Mask the mosaic data           
            # Step 1: Outside radar areas are set to NaN
            masked_rainfall = np.where(np.isnan(radar_mask), np.nan, rainfall_mosaic)

            # Step 2: Rainfall values less than 0.05 inside radar mask are set to 0 ตรงนี้เป็นฝนอ่อน ๆ ที่เกิดจากการแปลงฝนจาก dbz เป็น rain rate ด้วย zr และเกิดจากการ aggregrate จาก 15 นาที เป็น 1 ชั่วโมงตอนทำเรดาร์เดี่ยว
            masked_rainfall = np.where((masked_rainfall < 0.05) & (~np.isnan(radar_mask)), 0, masked_rainfall) 

            # Step 3: No rain (rainfall == 0) inside radar areas is set to 0
            masked_rainfall = np.where((masked_rainfall == 0) & (~np.isnan(radar_mask)), 0, masked_rainfall)

            # Save the results and plots
            if apply_mfb:
                #output_file = os.path.join(output_dir_mos_mfb, f'{time_hr}_rainfall_{mosaic_method}_{"mfb"}.tif')
                output_file = os.path.join(output_dir_mos_mfb, f'{time_hr}.tif')
                save_geotiff(output_file, masked_rainfall, transform, crs)
                plot_rainfall_map(masked_rainfall, radar_mask, transform, crs, basin_shapefile, output_dir_map, time_hr, 
                                  vmin=0., vmax=10, filename_suffix=f'_{mosaic_method}_{"mfb"}')
            else:
                #output_file = os.path.join(output_dir_mos_no_mfb, f'{time_hr}_rainfall_{mosaic_method}_{"no_mfb"}.tif')
                output_file = os.path.join(output_dir_mos_no_mfb, f'{time_hr}.tif')
                save_geotiff(output_file, masked_rainfall, transform, crs)
                plot_rainfall_map(masked_rainfall, radar_mask, transform, crs, basin_shapefile, output_dir_map, time_hr, 
                                  vmin=0., vmax=10, filename_suffix=f'_{mosaic_method}_{"no_mfb"}')

        except Exception as e:
            print(f"An error occurred during the mosaic process for {time_hr}: {str(e)}")
            import traceback
            traceback.print_exc()

    print("Mosaic processing completed for all time steps.")


if __name__ == "__main__":
    # Run the main function twice, once with MFB and once without
    main(mosaic_method='mean', apply_mfb=True)  # With MFB
    main(mosaic_method='mean', apply_mfb=False)  # Without MFB
    main(mosaic_method='max', apply_mfb=True)  # With MFB
    main(mosaic_method='max', apply_mfb=False)  # Without MFB


Mosaic report saved to ../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/1mean_mos/0mosaics\mosaic_report.csv
Processing time: 2018071507
Processing time: 2018071508
Processing time: 2018071509
Processing time: 2018071510
Processing time: 2018071511
Processing time: 2018071512
Processing time: 2018071513
Processing time: 2018071514
Processing time: 2018071515
Processing time: 2018071516
Processing time: 2018071517
Processing time: 2018071518
Processing time: 2018071519
Processing time: 2018071520
Processing time: 2018071521
Processing time: 2018071522
Processing time: 2018071523
Processing time: 2018071600
Processing time: 2018071601
Processing time: 2018071602
Processing time: 2018071603
Processing time: 2018071604
Processing time: 2018071605
Processing time: 2018071606
Processing time: 2018071607
Processing time: 2018071608
Processing time: 2018071609
Processing time: 2018071610
Processing time: 2018071611
Processing time: 2018071612
Processing time: 2018071613
P

In [9]:
'''
# เวิร์ค เป็นต้นแบบได้ ในการ apply MFB  mask เรดาร์ได้แล้ว สร้างโฟลเดอร์ผลลัพธ์เองได้ รันข้อมูลจริง sontihn
import os
import csv
import glob
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.transform import from_bounds
from rasterio.crs import CRS
from rasterio.features import rasterize
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import CRS, Transformer, Geod

def get_input_files(base_dir):
    radar_files = {
        'CHN': [], 'CMP': [], 'CRI': [], 'KKN': [], 'KRB': [], 'LMP': [],
        'NRT': [], 'PHS': [], 'SNK': [], 'STP': [], 'SVP': [], 'PKT': []
    }
    
    for radar in radar_files.keys():
        pattern = os.path.join(base_dir, f'00outp_cappi_2km_{radar}', '*.tif')
        files = sorted(glob.glob(pattern))
        radar_files[radar] = files
    
    return radar_files

def generate_mosaic_report(radar_files, output_dir, expected_radars=12):
    report_data = []
    all_timestamps = set()
    
    for radar_list in radar_files.values():
        all_timestamps.update(os.path.basename(f).split('.')[0] for f in radar_list)
    
    all_timestamps = sorted(all_timestamps)
    
    for timestamp in all_timestamps:
        available_radars = [radar for radar, file_list in radar_files.items() 
                            if any(timestamp in f for f in file_list)]
        
        report_data.append({
            'timestamp': timestamp,
            'available_radars': ','.join(available_radars),
            'total_radars': len(available_radars),
            'expected_radars': expected_radars,
            'missing_radars': expected_radars - len(available_radars)
        })
    
    report_file = os.path.join(output_dir, 'mosaic_report.csv')
    with open(report_file, 'w', newline='') as csvfile:
        fieldnames = ['timestamp', 'available_radars', 'total_radars', 'expected_radars', 'missing_radars']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in report_data:
            writer.writerow(row)
    
    print(f"Mosaic report saved to {report_file}")
    return report_data, all_timestamps

def reproject_to_geographic(src, dst_crs=CRS.from_epsg(4326)):
    dst_transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    
    reprojected_data = np.full((height, width), np.nan, dtype=np.float32)
    reproject(
        source=rasterio.band(src, 1),
        destination=reprojected_data,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )
    
    return reprojected_data, dst_transform, dst_crs

def create_geodetic_radar_buffers(radar_dict, active_radars, radius_km, crs):
    radar_data = []
    geod = Geod(ellps="WGS84")
    azimuths = np.linspace(0, 360, 361)
    
    for radar in active_radars:
        info = radar_dict[radar]
        lon, lat, _ = info['coords']
        buffer_points = [geod.fwd(lon, lat, azimuth, radius_km * 1000)[:2] for azimuth in azimuths]
        radar_data.append({'radar': radar, 'geometry': Polygon(buffer_points)})
    
    gdf_radars = gpd.GeoDataFrame(radar_data, crs=crs)
    return gdf_radars

def dbz_to_rainfall(dbz, zr_relation='marshall-palmer'):
    zr_relations = {
        'marshall-palmer': (200, 1.6),
        'rosenfeld-tropical': (250, 1.2),
        'summer-deep-convective': (300, 1.4)
    }
    
    if zr_relation not in zr_relations:
        raise ValueError(f"Unknown Z-R relation: {zr_relation}. Available options are: {', '.join(zr_relations.keys())}")
    
    a, b = zr_relations[zr_relation]
    Z = 10 ** (dbz / 10)  # Convert dBZ to Z
    R = (Z / a) ** (1 / b)
    return R

def load_mfb_values(file_path, threshold='1.0'):
    mfb_values = {}
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row['Threshold'] == threshold:
                radar = row['Radar']
                zr_relation = row['ZR_Relation']
                mfb = float(row['MFB']) if row['MFB'] else 1.0
                
                if radar not in mfb_values:
                    mfb_values[radar] = {}
                mfb_values[radar][zr_relation] = mfb
    #print('mfb_values>>>> threshold : ', threshold, mfb_values)
    return mfb_values

def apply_mfb_adjustment(radar_data, radar_code, zr_relation, mfb_values):
    if radar_code not in mfb_values or zr_relation not in mfb_values[radar_code]:
        print(f"Warning: No MFB value found for radar {radar_code} and Z-R relation {zr_relation}. Using unadjusted data.")
        return radar_data
    
    mfb = mfb_values[radar_code][zr_relation]
    #print('mfb_values[radar_code][zr_relation]>>>>//', radar_code, ' ,', zr_relation, ' ,',  mfb_values[radar_code][zr_relation])
    
    # Apply MFB adjustment only for rainfall values greater than 0
    adjusted_data = np.where(radar_data > 0, radar_data * mfb, radar_data)
    
    return adjusted_data

def calculate_mosaic_extent_geo(reprojected_data):
    #ax.set_xlim(96.15, 106.5)
    #ax.set_ylim(4.25, 22.15)
    
    #all_bounds = [rasterio.transform.array_bounds(data.shape[0], data.shape[1], transform) 
    #              for data, transform, _, _ in reprojected_data]
    
    #left = min(bound[0] for bound in all_bounds)
    #bottom = min(bound[1] for bound in all_bounds)
    #right = max(bound[2] for bound in all_bounds)
    #top = max(bound[3] for bound in all_bounds)
    left = 96.15
    bottom = 4.25
    right = 106.5
    top = 22.15
    
    
    return (left, bottom, right, top)

def calculate_rainfall_mosaic(reprojected_data, resolution, method='mean', zr_relation='marshall-palmer', mfb_values=None, apply_mfb=False):
    mosaic_bounds = calculate_mosaic_extent_geo(reprojected_data)
    width = int((mosaic_bounds[2] - mosaic_bounds[0]) / resolution)
    height = int((mosaic_bounds[3] - mosaic_bounds[1]) / resolution)
    dst_transform = from_bounds(*mosaic_bounds, width, height)

    if method == 'mean':
        summed_rainfall_data = np.zeros((height, width), dtype=np.float64)
        count_valid_data = np.zeros((height, width), dtype=np.float64)
    elif method == 'max':
        max_rainfall_data = np.full((height, width), -np.inf, dtype=np.float64)
    else:
        raise ValueError("Method must be either 'mean' or 'max'")

    combined_mask = np.zeros((height, width), dtype=bool)

    for data, src_transform, _, radar_code in reprojected_data:
        resampled = np.full((height, width), np.nan, dtype=np.float32)
        reproject(
            source=data,
            destination=resampled,
            src_transform=src_transform,
            dst_transform=dst_transform,
            src_crs=CRS.from_epsg(4326),
            dst_crs=CRS.from_epsg(4326),
            resampling=Resampling.nearest
        )
        rainfall_data = dbz_to_rainfall(resampled, zr_relation)
        
        if apply_mfb and mfb_values is not None:
            rainfall_data = apply_mfb_adjustment(rainfall_data, radar_code, zr_relation, mfb_values)

        radar_mask = ~np.isnan(rainfall_data)
        combined_mask |= radar_mask

        if method == 'mean':
            summed_rainfall_data = np.nansum([summed_rainfall_data, rainfall_data], axis=0)
            count_valid_data += radar_mask.astype(np.float64)
        elif method == 'max':
            max_rainfall_data = np.fmax(max_rainfall_data, rainfall_data)

    if method == 'mean':
        result_rainfall_data = np.divide(summed_rainfall_data, count_valid_data, where=count_valid_data > 0)
    elif method == 'max':
        result_rainfall_data = np.where(max_rainfall_data == -np.inf, np.nan, max_rainfall_data)

    return result_rainfall_data, dst_transform, CRS.from_epsg(4326)

def apply_threshold_and_mask(data, radar_mask, threshold=0.1):
    result = np.full_like(data, np.nan, dtype=np.float32)
    
    mask_within_coverage = ~np.isnan(radar_mask)
    
    result[mask_within_coverage] = np.where(data[mask_within_coverage] >= threshold, 
                                            data[mask_within_coverage], 
                                            0)
    
    return result

def save_geotiff(output_file, data, transform, crs):
    with rasterio.open(
        output_file, 'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(data, 1)

def plot_rainfall_map(data, radar_mask, transform, crs, basin_shapefile, output_dir, time_hr, vmin=0, vmax=5, filename_suffix=''):
    """
    Plot rainfall estimate map with jet colormap, grey for 0 mm inside radar coverage,
    and white outside radar coverage.
    """
    # Create a custom colormap
    cmap = plt.cm.jet
    n_bins = 256
    jet_colors = cmap(np.linspace(0, 1, n_bins))
    
    # Set the first color (for 0 mm) to grey
    jet_colors[0] = [0.5, 0.5, 0.5, 1]
    
    custom_cmap = ListedColormap(jet_colors)
    custom_cmap.set_bad('white')  # For NaN values (outside radar mask)

    # Create a masked array for the rainfall data
    masked_data = np.ma.masked_where(np.isnan(radar_mask), data)

    # Plotting the data
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Define the extent for the plot
    extent = rasterio.transform.array_bounds(data.shape[0], data.shape[1], transform)
    extent = [extent[0], extent[2], extent[1], extent[3]]  # [min_lon, max_lon, min_lat, max_lat]

    # Plot the rainfall data
    im = ax.imshow(masked_data, cmap=custom_cmap, extent=extent, origin='upper', 
                   vmin=vmin, vmax=vmax)

    # Load and plot the basin shapefile
    gdf = gpd.read_file(basin_shapefile)
    gdf = gdf.to_crs(crs)
    gdf.boundary.plot(ax=ax, color='black', linewidth=1)

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, extend='max')
    cbar.set_label('Rainfall Intensity (mm/hr)')

    ax.set_title(f'Radar Rainfall Estimate - {time_hr}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Set extent to the specified values
    ax.set_xlim(96.15, 106.5)
    ax.set_ylim(4.25, 22.15)
    
    plt.tight_layout()
    
    # Save plot
    plt.savefig(f'{output_dir}/{time_hr}_rainfall{filename_suffix}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up memory

'''    
def plot_radar_coverage_mask(radar_mask, transform, output_dir, filename='radar_coverage_mask.png'):
    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Define the extent for the plot
    extent = rasterio.transform.array_bounds(radar_mask.shape[0], radar_mask.shape[1], transform)
    extent = [extent[0], extent[2], extent[1], extent[3]]

    # Plot the radar coverage mask
    im = ax.imshow(~np.isnan(radar_mask), cmap='binary', extent=extent, origin='upper')

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Radar Coverage')
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels(['No Coverage', 'Coverage'])

    # Set title and labels
    ax.set_title('Radar Coverage Mask')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Set plot limits (adjust these as needed to match your data)
    ax.set_xlim(96.15, 106.5)
    ax.set_ylim(4.25, 22.15)

    plt.tight_layout()
    plt.savefig(f'{output_dir}/{filename}', dpi=300, bbox_inches='tight')
    plt.show(fig)  # Close the figure to free up memory

    print(f"Radar coverage mask plot saved to {output_dir}/{filename}")    
'''
def main(mosaic_method='mean', apply_mfb=True):
    #base_dir = '../3output/0mosaic/0ztest/0processing_single_cappi/'
    base_dir = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0processing_single_cappi/'
    
    # Define output directories based on the mosaic method
    if mosaic_method == 'mean':
        output_dir_mos = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/1mean_mos/0mosaics'
        mosaic_num = '1'
    elif mosaic_method == 'max':
        output_dir_mos = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/2max_mos/0mosaics'
        mosaic_num = '2'

    output_dir_mos_mfb = os.path.join(output_dir_mos, '0mfb')
    output_dir_mos_no_mfb = os.path.join(output_dir_mos, '0no_mfb')

    # Here we add the number before mosaic_method using `mosaic_num`
    output_dir_map = f'../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/{mosaic_num}{mosaic_method}_mos/1map_plot'

    # Create necessary directories if they don't exist
    os.makedirs(output_dir_mos_mfb, exist_ok=True)
    os.makedirs(output_dir_mos_no_mfb, exist_ok=True)
    os.makedirs(output_dir_map, exist_ok=True)  # This ensures that the map plot directory is created
   
    resolution = 0.02  # Resolution in degrees
    
    radar_files = get_input_files(base_dir)
    basin_shapefile = '../1data/1GIS/0base_map_gis/MainBasin_ONWR_Law_WGS84Geo.shp'
    
    all_radars = {
        "CHN": {'coords': (100.191263, 15.157852, 40.0), 'el': 0.5},
        "CMP": {'coords': (99.188203, 10.493099, 32.0), 'el': 0.0},
        "CRI": {'coords': (99.881593, 19.961471, 444.0), 'el': 1.08},
        "KKN": {'coords': (102.785881, 16.4625, 217.0), 'el': 0.47},
        "KRB": {'coords': (98.97806, 8.101389, 52.0), 'el': 0.8},
        "LMP": {'coords': (99.041701, 18.565399, 320.0), 'el': 0.48},
        "NRT": {'coords': (101.825165, 6.426888, 33.0), 'el': 0.5},
        "PHS": {'coords': (100.217964, 16.775408, 72.0), 'el': 0.47},
        "PKT": {'coords': (98.329444, 8.133611, 281.0), 'el': 0.08},
        "SNK": {'coords': (104.132591, 17.156363, 196.0), 'el': 0.47},
        "STP": {'coords': (100.459996, 7.449996, 33.0), 'el': 0.5},
        "SVP": {'coords': (100.7675, 13.686389, 28.0), 'el': 0.7},
    }

    report_data, all_timestamps = generate_mosaic_report(radar_files, output_dir_mos, expected_radars=12)

    mfb_values = None
    if apply_mfb:
        mfb_file_path = './0Zprocessing_data/0analysis_results/combined_results_threshold_1.0.csv'
        mfb_values = load_mfb_values(mfb_file_path)

    for time_hr in all_timestamps:
        print(f"Processing time: {time_hr}")
        try:
            current_radar_files = {
                radar: next((f for f in files if time_hr in f), None)
                for radar, files in radar_files.items()
            }
            current_radar_files = {k: v for k, v in current_radar_files.items() if v is not None}

            if not current_radar_files:
                print(f"No radar files available for {time_hr}. Skipping.")
                continue

            active_radars = list(current_radar_files.keys())

            reprojected_data = []
            for radar, radar_file in current_radar_files.items():
                with rasterio.open(radar_file) as src:
                    data, transform, crs = reproject_to_geographic(src)
                    reprojected_data.append((data, transform, crs, radar))

            rainfall_mosaic, transform, crs = calculate_rainfall_mosaic(
                reprojected_data, resolution, method=mosaic_method, 
                zr_relation='rosenfeld-tropical', mfb_values=mfb_values, apply_mfb=apply_mfb
            )

            radar_buffers_geo = create_geodetic_radar_buffers(all_radars, active_radars, 240, "EPSG:4326")

            radar_mask = rasterize(
                [(geom, 1) for geom in radar_buffers_geo.geometry],
                out_shape=rainfall_mosaic.shape,
                transform=transform,
                fill=np.nan,
                dtype=np.float32
            )
            # Mask the mosaic data           
            # Step 1: Outside radar areas are set to NaN
            masked_rainfall = np.where(np.isnan(radar_mask), np.nan, rainfall_mosaic)

            # Step 2: Rainfall values less than 0.05 inside radar mask are set to 0 ตรงนี้เป็นฝนอ่อน ๆ ที่เกิดจากการแปลงฝนจาก dbz เป็น rain rate ด้วย zr และเกิดจากการ aggregrate จาก 15 นาที เป็น 1 ชั่วโมงตอนทำเรดาร์เดี่ยว
            masked_rainfall = np.where((masked_rainfall < 0.05) & (~np.isnan(radar_mask)), 0, masked_rainfall) 

            # Step 3: No rain (rainfall == 0) inside radar areas is set to 0
            masked_rainfall = np.where((masked_rainfall == 0) & (~np.isnan(radar_mask)), 0, masked_rainfall)

            # Save the results and plots
            if apply_mfb:
                output_file = os.path.join(output_dir_mos_mfb, f'{time_hr}_rainfall_{mosaic_method}_{"mfb"}.tif')
                save_geotiff(output_file, masked_rainfall, transform, crs)
                plot_rainfall_map(masked_rainfall, radar_mask, transform, crs, basin_shapefile, output_dir_map, time_hr, 
                                  vmin=0., vmax=10, filename_suffix=f'_{mosaic_method}_{"mfb"}')
            else:
                output_file = os.path.join(output_dir_mos_no_mfb, f'{time_hr}_rainfall_{mosaic_method}_{"no_mfb"}.tif')
                save_geotiff(output_file, masked_rainfall, transform, crs)
                plot_rainfall_map(masked_rainfall, radar_mask, transform, crs, basin_shapefile, output_dir_map, time_hr, 
                                  vmin=0., vmax=10, filename_suffix=f'_{mosaic_method}_{"no_mfb"}')

        except Exception as e:
            print(f"An error occurred during the mosaic process for {time_hr}: {str(e)}")
            import traceback
            traceback.print_exc()

    print("Mosaic processing completed for all time steps.")


if __name__ == "__main__":
    # Run the main function twice, once with MFB and once without
    main(mosaic_method='mean', apply_mfb=True)  # With MFB
    main(mosaic_method='mean', apply_mfb=False)  # Without MFB
    main(mosaic_method='max', apply_mfb=True)  # With MFB
    main(mosaic_method='max', apply_mfb=False)  # Without MFB
'''

Mosaic report saved to ../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/1mean_mos/0mosaics\mosaic_report.csv
Processing time: 2018071507
Processing time: 2018071508
Processing time: 2018071509
Processing time: 2018071510
Processing time: 2018071511
Processing time: 2018071512
Processing time: 2018071513
Processing time: 2018071514
Processing time: 2018071515
Processing time: 2018071516
Processing time: 2018071517
Processing time: 2018071518
Processing time: 2018071519
Processing time: 2018071520
Processing time: 2018071521
Processing time: 2018071522
Processing time: 2018071523
Processing time: 2018071600
Processing time: 2018071601
Processing time: 2018071602
Processing time: 2018071603
Processing time: 2018071604
Processing time: 2018071605
Processing time: 2018071606
Processing time: 2018071607
Processing time: 2018071608
Processing time: 2018071609
Processing time: 2018071610
Processing time: 2018071611
Processing time: 2018071612
Processing time: 2018071613
P